# Data Extraction, Transformation, and Loading: PostgreSQL to AWS S3

This notebook provides a step-by-step guide to perform the necessary steps to extract data from a PostgreSQL database, transform it, and load it into an Amazon S3 bucket. Specifically, you will:

1. **Connect to the PostgreSQL Database**: Utilize the `psycopg2` library to establish a connection to the database and execute SQL queries to retrieve data.

2. **Convert SQL Query Results into a Pandas DataFrame**: Leverage the `pandas` library to convert the retrieved data into a DataFrame for easier handling and manipulation.

3. **Transform the Data**: Apply ETL (Extract, Transform, Load) operations on the DataFrame to clean and prepare the data for further use.

4. **Upload the Transformed Data to S3**: Use the `boto3` library to send the prepared data to an Amazon S3 bucket for storage.

This notebook serves as a practical tool to guide you through these steps, ensuring a seamless data extraction, transformation, and loading process. Let's dive in!


<br><br>

# Loading Libraries and Packages

Before we begin with the data extraction and transformation process, we need to load the necessary libraries and packages. This section will ensure that we have all the tools required to connect to the PostgreSQL database, manipulate the data, and interact with AWS S3.


### Checking and Installing Required Packages
To ensure that our environment has all the necessary packages for this notebook, we will first check if each package is installed. If any of the required packages are missing, we will download and install them. This step is crucial to avoid any interruptions during our data extraction, transformation, and loading process. Run the cell below by hitting **Control + Enter**.

In [1]:
import subprocess
import sys

# Function to install packages
def install_package(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# List of required packages
required_packages = [
    'requests',
    'traceback',
    'os',
    'boto3',
    'pandas',
    'dotenv',
    'datetime',
    'psycopg2'
]

# Checking and installing packages
for package in required_packages:
    try:
        __import__(package)
    except ImportError:
        print(f"{package} not found. Installing...")
        install_package(package)

print("All required packages are installed.")

All required packages are installed.


### Explanation of Packages

- **requests**: A library for making HTTP requests to interact with web services and APIs.
- **traceback**: A module for printing or retrieving a stack traceback, useful for debugging and error handling.
- **os**: A module that provides a way to interact with the operating system, allowing us to access environment variables, file paths, and more.
- **boto3**: The AWS SDK for Python, which allows us to interact with Amazon Web Services (AWS), such as S3, EC2, and more.
- **pandas**: A powerful data manipulation and analysis library that provides data structures like DataFrames for handling and analyzing data.
- **dotenv**: A library for reading key-value pairs from a `.env` file and loading them into the environment, useful for managing configuration and credentials.
- **datetime**: A module for working with dates and times, enabling us to manipulate and format date and time data.
- **psycopg2**: A PostgreSQL adapter for Python that allows us to connect to a PostgreSQL database and execute SQL queries.

Now let's import these libraries and packages to ensure we have everything we need for our data processing workflow.


In [2]:
import requests
import traceback
import os
import boto3
import pandas as pd
from dotenv import load_dotenv
from datetime import datetime
import psycopg2

### Importing Packages and Functions from Local Python Files

In this section, we will import any additional packages and custom functions from local Python files. This step ensures that all necessary modules and functions specific to our project are available for use.


In [3]:
from query_package import set_project
from query_package import build_query_package

from rds_connection import connect_rds
from rds_processor import RDS

from s3_bucket_util import update_active_data
from s3_bucket_util import add_archive

# Loading Environment Variables

In this section, we will explain how to load environment variables from a `.env` file. These variables are used to securely store sensitive information such as tokens, usernames, passwords, and paths that your application requires to function. This approach helps to keep such details out of your source code, enhancing security and making it easier to manage configurations.

In [4]:
#Load Environment Variables
access = os.getenv('ACCESS')    # AWS access token
secret = os.getenv('SECRET')    # AWS secret token
username = os.getenv('USER')    # AWS RDS username
password = os.getenv('PASS')    #AWS RDS password
server = os.getenv('SERVER')    # AWS RDS server
db = os.getenv('DB')    # AWS RDS database
bucket = os.getenv('BUCKET')    # AWS RDS bucket name

<br><br>

# Forming the SQL Query

## Input Project and Questions Information

To begin, we need to gather the necessary project details and specific questions we aim to answer. This information will help us build an effective SQL query to retrieve the required data from the PostgreSQL database.

### Selecting the Project
The first step is to select the project you are working on. This can either be **Helene** or **Milton**. Depending on the project selected, the structure and requirements of the SQL query may vary. Make sure to choose the project that aligns with your current focus to ensure accurate data retrieval and analysis.
Once the project is selected, we will proceed to define the specific questions or objectives that the SQL query aims to address. This will guide the formulation of the query and ensure that the retrieved data is relevant and useful for our analysis.

> **Note:** If project names or codes need to be updated, this must be done in the Python file called `query_package` in the folder. The section that must be updated is located near the top and is called `project_dictionary`.




In [5]:
project = set_project("Helene")

You've selected  Helene  ID:34
If the name and id are correct please proceed.  If not, please review and update the project dicitonary in query_package.py


<div style="height: 10px;"></div>

### Adding Questions

Below, you will find the format for each question that you will want to pull from the SQL Database. Each question will result in a field in the overall table. This format ensures that all necessary information is included for building the SQL query. To add more questions, simply copy and paste the blank first question into the list in the cell below.

#### Explanation of Dictionary Entries

- **name**: This specifies what we want to name the field in the final dataset and will be used for joining when performing the query. Names of fields are recommended to be all lower case with spaces replaced by **"_"**. For example, 'hotel_name' or 'room_rate'.
  
- **single_or_repeat**: Indicates whether the question pertains to a single-entry ('SINGLE') or repeated entries ('REPEAT') within the PostgreSQL database.
  
- **data_source**: Refers to the table source within the PostgreSQL database where the question data can be found. For example, an answer containing integers will be stored in **application_data_numberanswer**.
  
- **question_id**: A unique identifier for the question, typically an integer that corresponds to the specific question in the database.
  
- **fields**: This specifies how we will rename the field to its official name when finally added into the overall dataset. The left side of the pair needs to be the name of the field stored in the question table within the **data_source** section, and the other side will be the name of the field we want in our final table. For example, in the **application_data_textboxanswer** table in the PostgreSQL database, answers are stored in a field called **value**. Therefore, **value** will be on the left side of the pair, but when we add this to the final dataset, we want a more unique name, so we add the desired field name to the right side of the pair. When we get a DataFrame back, the field is now named **hotel_name**. This must be done for all fields that we want to pull from the data source table in the PostgreSQL database. View the two examples below, and be sure to notice the difference in the fields section between the two questions.  It is recommended that you name this field after the name of the join in the first part of the package.



#### EXAMPLE
```python
questions = [
    {
        'name': 'hotel_name',
        'single_or_repeat': 'SINGLE',
        'data_source': 'application_data_textboxanswer',
        'question_id': 1015,
        'fields': [{'value': 'hotel_name'}]
    },

    {
        'name': 'hotel_address',
        'single_or_repeat': 'REPEATING',
        'question_id': 1016,
        'data_source': 'application_data_addressanswer',
        'fields': {
            'line1': 'hotel_address_line_1',
            'line2': 'hotel_address_line_2',
            'city': 'hotel_city',
            'state': 'hotel_state',
            'zip': 'hotel_zip'
        }
    }
]



#### Fill Out the Question List

To fill out the questions package, simply fill out each section with the information needed.  If more questions are needed, place a comm behind the closing **"}"**, then copy and paste a blank package of information below, make sure not to place it below the **"]"** at the very bottom of the cell. Repeat untill you have the total list of fields you want pulled from the PostgreSQL database. After you are done, hit **CONTROL + ENTER** to finalize.  In thi cell below the questions package is an example for pulling a database that focuses on hotel information.  You can use this as a reference while building out your own questions package.  Hit the **Click to Expand** to view this example

In [6]:
questions = [
    
    {
        'name': '',
        'single_or_repeat': '',
        'data_source': '',
        'question_id': 0,
        'fields': [{'': ''}]
    },

    {
        'name': '',
        'single_or_repeat': '',
        'data_source': '',
        'question_id': 0,
        'fields': [{'': ''}]
    },

    {
        'name': '',
        'single_or_repeat': '',
        'data_source': '',
        'question_id': 0,
        'fields': [{'': ''}]
    }




]

In [7]:
questions = [
    
    {'name': 'hotel_name',
    'single_or_repeat': 'SINGLE',
    'data_source': 'application_data_textboxanswer',
    'question_id': 1015,
    'fields' : [{'value':'hotel_name'}]
    },


    {'name': 'hotel_address',
    'single_or_repeat': 'REPEATING',
    'question_id': 1016,
    'data_source': 'application_data_addressanswer',
    'fields': [{'line1':'hotel_address_line_1',
                'line2': 'hotel_address_line_2',
                'city' : 'hotel_city',
                'state': 'hotel_state',
                'zip' : 'hotel_zip'}]
    },


    {'name': 'hotel_status',
    'single_or_repeat': 'REPEATING',
    'data_source': 'application_data_singleselectanswer',
    'question_id': 1013,
    'fields': [{'value':'hotel_status'}]
    },


    {'name': 'license_in',
    'single_or_repeat': 'REPEATING',
    'data_source': 'application_data_dateanswer',
    'question_id': 1021,
    'fields': [{'value':'license_in'}]
    },


    {'name': 'license_out',
    'single_or_repeat': 'REPEATING',
    'data_source': 'application_data_dateanswer',
    'question_id': 1022,
    'fields': [{'value':'license_out'}]
    },


    {'name': 'total_in_household',
    'single_or_repeat': 'SINGLE',
    'data_source': 'application_data_numberanswer',
    'question_id': 596,
    'fields': [{'value':'total_in_household'}]
    },


    {'name': 'active_bookings',
    'single_or_repeat': 'SINGLE',
    'data_source': 'application_data_numberanswer',
    'question_id': 1010,
    'fields': [{'value':'active_bookings'}]
    },

    {'name': 'pathway_determination',
    'single_or_repeat': 'SINGLE',
    'data_source': 'application_data_singleselectanswer',
    'question_id': 632,
    'fields': [{'value':'pathway_determination'}]
    }
]

#### EXAMPLE HOTEL SUMMARIES QUESTION LIST

<details> <summary>Click to expand</summary>

```python
questions = [
    
    {'name': 'hotel_name',
    'single_or_repeat': 'SINGLE',
    'data_source': 'application_data_textboxanswer',
    'question_id': 1015,
    'fields' : [{'value':'hotel_name'}]
    },


    {'name': 'hotel_address',
    'single_or_repeat': 'REPEATING',
    'question_id': 1016,
    'data_source': 'application_data_addressanswer',
    'fields': [{'line1':'hotel_address_line_1',
                'line2': 'hotel_address_line_2',
                'city' : 'hotel_city',
                'state': 'hotel_state',
                'zip' : 'hotel_zip'}]
    },


    {'name': 'hotel_status',
    'single_or_repeat': 'REPEATING',
    'data_source': 'application_data_singleselectanswer',
    'question_id': 1013,
    'fields': [{'value':'hotel_status'}]
    },


    {'name': 'license_in',
    'single_or_repeat': 'REPEATING',
    'data_source': 'application_data_dateanswer',
    'question_id': 1021,
    'fields': [{'value':'license_in'}]
    },


    {'name': 'license_out',
    'single_or_repeat': 'REPEATING',
    'data_source': 'application_data_dateanswer',
    'question_id': 1022,
    'fields': [{'value':'license_out'}]
    },


    {'name': 'total_in_household',
    'single_or_repeat': 'SINGLE',
    'data_source': 'application_data_numberanswer',
    'question_id': 596,
    'fields': [{'value':'total_in_household'}]
    },


    {'name': 'active_bookings',
    'single_or_repeat': 'SINGLE',
    'data_source': 'application_data_numberanswer',
    'question_id': 1010,
    'fields': [{'value':'active_bookings'}]
    },

    {'name': 'pathway_determination',
    'single_or_repeat': 'SINGLE',
    'data_source': 'application_data_singleselectanswer',
    'question_id': 632,
    'fields': [{'value':'pathway_determination'}]
    }
]

</details>

<div style="height: 20px;"></div>

### Build the Query

Now we'll put all of this together, run the cell below to build the entire query package that we will be sending to the postgres database.  After the function is run, you can view the entire package by running the cell that is labeled **View Package**.  Remember, we need both the project and the questions in order build the query, if you haven't hit **CONTROL + ENTER** on these cells, its time to do it now.


In [8]:
query_package = build_query_package(project, questions)

##### VIEW PACKAGE

In [9]:
query_package

{'source': {'table': 'applications_application',
  'name': 'app',
  'fields': [{'application_number': 'application_number',
    'created_at': 'created_at'}],
  'project': 34,
  'order': 'id'},
 'join_list': [{'name': 'hotel_name',
   'single_or_repeat': 'SINGLE',
   'data_source': 'application_data_textboxanswer',
   'question_id': 1015,
   'fields': [{'value': 'hotel_name'}]},
  {'name': 'hotel_address',
   'single_or_repeat': 'REPEATING',
   'question_id': 1016,
   'data_source': 'application_data_addressanswer',
   'fields': [{'line1': 'hotel_address_line_1',
     'line2': 'hotel_address_line_2',
     'city': 'hotel_city',
     'state': 'hotel_state',
     'zip': 'hotel_zip'}]},
  {'name': 'hotel_status',
   'single_or_repeat': 'REPEATING',
   'data_source': 'application_data_singleselectanswer',
   'question_id': 1013,
   'fields': [{'value': 'hotel_status'}]},
  {'name': 'license_in',
   'single_or_repeat': 'REPEATING',
   'data_source': 'application_data_dateanswer',
   'question

<br></br>

## Connecting to PostgreSQL Database and Creating a Pandas DataFrame

In this section, we will take the recently created query package and connect to our PostgreSQL database. Using the package, we will pull data from the database and convert it into a pandas DataFrame for further analysis.

### Steps to Connect and Retrieve Data

1. **Establish Database Connection**
   - Next, we will establish a connection to our PostgreSQL database using the connection credentials.

2. **Execute Query and Retrieve Data**
   - Using the query package, we will execute SQL queries to pull data from the database.

3. **Convert Data to Pandas DataFrame**
   - Finally, we will convert the retrieved data into a pandas DataFrame for ease of data manipulation and analysis.


### Connect to the PostgreSQL Database

Below we'll use a function in the **rds_connection.py** file that will use the username, password, database name, and server named stored in the enviroment file to acess the PostgreSQL database.  Run the cell below to connect, if an error is returned, review the credentials in the .env file and try again.

In [10]:
conn, cursor = connect_rds(username, password, db, server)

<div style="height: 10px;"></div>

### Create RDS Connector Tool and Pull Dataframe

Now we'll pull something called the RDS Connector Tool from a local file called **rds_processor.py**, this is a packaged designed to take in the query package and connection to process and build the dataframe.  The table will be stored in an item called **df**. During the building of the table, duplicates will be removed, you can view any of the duplicates by calling the **duplicate** item.  If the table is empty, then no duplicates were found.  You can view the dataframe by hitting **CONTROL + ENTER** on the cell labeled **VIEW DATA**

In [11]:
# Create the Connector
rds = RDS(conn, cursor, query_package) 

# Store the Dataframe
df = rds.df

# Store Duplicates
duplicates = rds.duplicates

##### VIEW DATA

In [12]:
df

,application_number,created_at,hotel_name,hotel_address_line_1,hotel_address_line_2,hotel_city,hotel_state,hotel_zip,hotel_status,license_in,license_out,total_in_household,active_bookings,pathway_determination
0,KQ6625DD77,2024-09-27 18:40:42.775742+00:00,None,None,None,None,None,None,None,None,None,2.00,None,Hotel
1,B2B9MZYTQL,2024-10-01 18:06:55.607877+00:00,,,,,FL,,Rejected,None,None,2.00,None,Commercial Site Placement
2,NY6LC7XV4W,2024-10-01 19:10:39.192189+00:00,None,None,None,None,None,None,None,None,None,None,None,
3,4VD5687DFG,2024-10-01 20:01:35.440307+00:00,,,,,FL,,Rejected,None,None,3.00,1.00,Hotel
4,2Y9YFK89FM,2024-10-01 20:43:52.325004+00:00,None,None,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8353,3RZX6PVUBD,2024-12-20 15:07:08.057391+00:00,None,None,None,None,None,None,None,None,None,None,None,None
8354,NJA9WGVT9M,2024-12-20 15:10:57.757964+00:00,None,None,None,None,None,None,None,None,None,None,None,None
8355,U476QS7F9E,2024-12-20 15:23:36.200563+00:00,None,None,None,None,None,None,None,None,None,None,None,None
8356,HT8Q6LM3B6,2024-12-20 16:08:29.894267+00:00,None,None,None,None,None,None,None,None,None,1.00,None,None


<br></br>

## DataFrame Modification Section

In this section, you can modify the DataFrame as needed. Use the cells below to perform any necessary transformations, cleaning, or manipulations. Ensure that the final form of your DataFrame is saved as `df` for consistency and further use.


In [13]:
########    PERFORM ETL WITH PANDAS    #########

<br></br>

## Loading DataFrame to S3 Bucket

In this final step, we will load the processed DataFrame to an AWS S3 bucket. This process involves saving the DataFrame as a file in the specified S3 bucket's active folder, and additionally making a copy of this file in the archive folder. This ensures that the most recent data is available in the active location, while also maintaining a historical record of previous data versions in the archive.


### Store S3 Bucket Information

In this section you will find the S3 Bucket that we will be sumitting the dataframe to and storing that information for later use.  To do this you will need the name of the project folder you are trying to submit to followed by a **"/"**.  You will also need to enter the name of the bucket, as of right now, it should always be **reporting-external**, but if this changes, it will have to be updated below.  Also before this step, make sure that you have correctly set up the S3 Bucket to receive data, if you havent done this, follow the tutorial **Set Up S3 Bucket for Data Upload**


##### EXAMPLE

```python
bucket = 'reporting-external'
project_folder = 'Example_for_Tutorial/'

In [14]:
# Set Bucket Name
bucket = 'reporting-external'

#Set Project Folder
project_folder = 'Example_for_Tutorial/'


<div style="height: 10px;"></div>

## Creating a Boto3 Client for S3

In this section, we will create a Boto3 client for AWS S3. The Boto3 client is essential for interacting with S3 services, allowing us to upload, download, and manage data stored in S3 buckets.  Credentials for this connection is stored in the .env file

### Storing Credentials Securely

To ensure that our AWS credentials (access key and secret key) are kept secure, we store them in a `.env` file. This approach prevents sensitive information from being hard-coded into the source code, enhancing the security of our application.

In [15]:
# Create a Boto3 client for S3
s3 = boto3.client(
    's3',
    aws_access_key_id=access,
    aws_secret_access_key=secret
)

<div style="height: 10px;"></div>

### Load DataFrame to Active S3 Bucket

In this section, we will take the final DataFrame from the previous step and load it into a folder within an S3 bucket. The function `update_active_data` will be used for this purpose. Here’s a breakdown of the parameters being passed into the function:

- **s3**: This is the Boto3 client for S3, which allows us to interact with the AWS S3 service.
- **bucket**: The name of the S3 bucket where the data will be uploaded.
- **project_folder**: The main project folder within the S3 bucket, which organizes your data.
- **active_folder**: The specific subfolder within the project folder where the active data will be stored. In this case, it is set to `'Active/'`.
- **file_name**: The name of the file that will be created in the active folder. This name can include dynamic elements such as the project folder name.
- **data**: The DataFrame containing the data to be uploaded. This should be the final form of the DataFrame, saved as `df`.

In [16]:
update_active_data(
    s3 = s3, 
    bucket = bucket, 
    project_folder = project_folder, 
    active_folder = 'Active/', 
    file_name = f"Active-{project_folder.replace("/", "").replace("_", "-")}.csv", 
    data = df
)

<div style="height: 10px;"></div>

### Load Archive to S3 Bucket

In this section, we will take a copy of the DataFrame and load it into an archive folder within the S3 bucket. The archive will maintain a maximum of 50 records in a rolling fashion, meaning new entries beyond 50 will replace the oldest ones. This setup ensures that you always have access to the most recent data while retaining a historical record up to a specified limit.

The function `add_archive` will be used for this purpose. Here’s a breakdown of the parameters being passed into the function:

- **s3**: This is the Boto3 client for S3, which allows us to interact with the AWS S3 service.
- **bucket**: The name of the S3 bucket where the data will be archived.
- **project_folder**: The main project folder within the S3 bucket, which organizes your data.
- **archive_folder**: The specific subfolder within the project folder where the archived data will be stored.
- **limit**: The maximum number of records to be retained in the archive. Once this limit is reached, older records will be replaced by newer ones.
- **versions**: This represents the archive history, such as `rds.archive`, which keeps track of previous data versions.

In [17]:
add_archive(
    s3 = s3, 
    bucket = bucket, 
    project_folder = project_folder, 
    archive_folder = 'Archive/', 
    limit = 50,
    archive_package = rds.archive
)